<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Overview" data-toc-modified-id="Overview-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Overview</a></span></li></ul></div>

<h3>Overview

This is part of a series of notebooks used to impute missing location data and create ***lat*** (latitude) and ***lon*** (longitude) columns. The ***location*** column of the original dataset is a dictionary with geocordinates and business address. Many of these location entries are null. Where not null, the coordinates are pulled from the location column. Where no coordinates are present, the a geocoding api is used to convert the address fields are to lat and lon. 

Note book 1.1d **Recombine and Finalize** merges the results of the two tranlsation process back into a single dataset.

In [2]:
import pandas as pd
import numpy as np
import json
import ast

import os
import sys
import re

module_path = os.path.abspath(os.path.join('./lib/'))

if module_path not in sys.path:   
    sys.path.append(module_path)

import datetime
from utilities import *

import matplotlib.pyplot as plt
import seaborn as sns


import geopandas as gpd
from geopandas.tools import geocode
from shapely.geometry import Point

import os
import sys
import re
import time

%matplotlib inline

In [3]:
def reload_by_loc():
    df = pd.read_csv('../tmp/converted_by_loc_then_address_final201920195843784591.csv', low_memory=False)
    #convert dates
    df['dba_start'] = pd.to_datetime(df['dba_start']) 
    df['dba_end'] = pd.to_datetime(df['dba_end'])
    df['loc_start'] = pd.to_datetime(df['loc_start'])
    df['loc_end'] = pd.to_datetime(df['loc_end'])
    return df

In [4]:
df_by_loc = reload_by_loc()

In [5]:
df_by_loc.shape

(143883, 27)

In [6]:
df_by_loc.columns

Index(['business_zip', 'certificate_number', 'city', 'dba_name',
       'full_business_address', 'lic', 'lic_code_description', 'location',
       'mail_city', 'mail_state', 'mail_zipcode', 'mailing_address_1',
       'naic_code', 'naic_code_description',
       'neighborhoods_analysis_boundaries', 'ownership_name', 'parking_tax',
       'state', 'supervisor_district', 'transient_occupancy_tax', 'ttxid',
       'dba_start', 'dba_end', 'loc_start', 'loc_end', 'lat', 'lon'],
      dtype='object')

In [7]:
def reload_by_addr():
    df = pd.read_csv('../tmp/converted_by_address_interim201920195832725116.csv', low_memory=False)
    #convert dates
    df['dba_start'] = pd.to_datetime(df['dba_start']) 
    df['dba_end'] = pd.to_datetime(df['dba_end'])
    df['loc_start'] = pd.to_datetime(df['loc_start'])
    df['loc_end'] = pd.to_datetime(df['loc_end'])
    return df

In [8]:
df_by_addr = reload_by_addr()

In [9]:
df_by_addr.shape

(51317, 27)

In [128]:
df = pd.concat([df_by_loc, df_by_addr])


In [129]:
df.shape

(195200, 27)

In [130]:
ts = get_timestamp()
df.to_csv(f'../tmp/recombined_interim{ts}.csv', index=False)